<a href="https://colab.research.google.com/github/maryantonopoulou/teliki_ergasia_EDDE2/blob/main/kathimerini_articles_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests # για να φέρνουμε τον html κώδικα των ιστοσελίδων
from bs4 import BeautifulSoup # για να βρίσκουμε μέσα στον html κώδικα, τα περιεχόμενα που μας ενδιαφέρουν
import pandas as pd # για να βάζουμε τα δεδομένα που συλλέγουμε σε dataframes

import time # για να κάνουμε παύσεις ανάμεσα στα αιτήματα που κάνουμε στον server του site ώστε να μην τον ρίξουμε ή να μην μας αποκλείσει
import random # για να δημιουργούμε ποικιλία στον χρόνο των παύσεων ώστε τα αιτήματα στον server να μοιάζουν με ανθρώπινα και όχι μηχανικά

from tqdm import tqdm # για να βάλω progress bar με το οποίο θα παρακολουθώ την πορεία του scraping

In [ ]:
def get_webpage_soup(url):
  # Ορισμός των headers για το requests
  headers = {
        "User-Agent": "Mozilla/5.0"
    }
  # Αίτημα στο server να φέρει τον html κώδικα από το url
  response = requests.get(url, headers=headers)

  # Έλεγχος της κατάστασης της σελίδας
  if response.status_code != 200:
    #print(f"Error: {response.status_code}")
    return None
  else:
    # Μετατροπή του πηγαίου κώδικα της ιστοσελίδας σε αναζητήσιμο κείμενο (soup)
    # print(f"{url} was scraped successfully")
    return BeautifulSoup(response.text, 'html.parser')

In [ ]:
def get_article_elements_from_soup(soup, article_url):
  # απομόνωση του block με το περιεχόμενο του άρθρου από όλη τη σούπα της σελίδας
  article_block = soup.find("article")
  # article url
  article_url = article_url # το παίρνω από το arguement του function
  # title
  try:
    title = article_block.find("h1").text.strip()
  except:
    try:
      title = soup.find("div", {"class": "container is-fullhd p-3"}).h1.text.strip() # προσθήκη για να πιάνει και τον τίτλο όταν βρίσκεται σε αυτό το div
    except:
      title=""
  # datetime
  try:
    datetime_str = article_block.find("time")["datetime"]
  except:
    try:
      datetime_str = soup.find("div", {"class": "container is-fullhd p-3"}).find("time")["datetime"] # προσθήκη για να πιάνει και το datetime όταν βρίσκεται σε αυτό το div
    except:
      datetime_str = ""
      date_str = ""
      time_str = ""
  if datetime_str != "":
    date_str = datetime_str.split("T")[0]
    time_str = datetime_str.split("T")[1].split("+")[0]
  # section
  try:
    section = article_block.find("span", class_=lambda c: c and "nx-single-category-title" in c).text.strip()
  except:
    section = ""
  # author
  try:
    author = article_block.find("span", class_=lambda c: c and "author" in c).text.strip()
  except:
    try:
      author = soup.find("div", {"class": "container is-fullhd p-3"}).find("a", {"rel": "author"}).text.strip() # προσθήκη
    except:
      author = ""
  # excerpt
  try:
    excerpt = article_block.find("div", {"class": "nx-excerpt pb-5"}).text.strip()
  except:
    try:
      excerpt = soup.find("div", {"class": "container is-fullhd p-3"}).find("div", {"class": "newsletter-description nx-excerpt my-5"}).text.strip() # προσθήκη
    except:
      excerpt=""
  # text body
  try:
    text_body_p_tagsL = article_block.find("div", {"id": "main-content"}).find_all("p")
    text_body_textsL = []
    for p_tag in text_body_p_tagsL:
      p_tag_text = p_tag.text.strip()
      text_body_textsL.append(p_tag_text)
    text_body = " ".join(text_body_textsL)
  except:
    text_body=""
  # feature image
  try:
    feat_img_link = article_block.find("div", {"class": "post-thumbnail"}).img["src"]
  except:
    feat_img_link=""
  # feature image caption
  try:
    image_caption = article_block.find("div", {"class": "img-caption mb-5"}).text.strip()
  except:
    image_caption=""

  # Δημιουργία λεξικού με τα data του άρθρου
  articleD = {"article_url": article_url, "title": title, "date": date_str, "time": time_str, "section": section, "author": author,
              "excerpt": excerpt, "text_body": text_body, "feat_img_link": feat_img_link, "feat_img_cation": image_caption}

  return articleD

In [ ]:
def scrape_kathimerini_articles_pages(teasers_df):
  # δημιουργία κενής λίστας στην οποία θα προσθέτουμε ένα-ένα τα λεξικά με τα δεδομένα των άρθρων που θα σκραπάρουμε
  all_articles_dataL = []
  # λούπα σε κάθε row του teasers_df για να παίρνουμε ένα-ένα τα urls των άρθρων και να το σκραπάρουμε
  # χρησιμοποιούμε το tqdm για να βλέπουμε σε ένα progress bar πού βρισκόμαστε και πόσος χρόνος απομένει ακόμα
  for idx, row in tqdm(teasers_df.iterrows(), total=teasers_df.shape[0], desc="Processing rows"):
    # παίρνουμε το url του άρθρου που θα σκραπάρουμε από τη στήλη "article_url" του row του teasers_df στο οποίο βρίσκεται η for loop
    article_url = row['article_url']
    # scrape της σελίδας του url του κάθε άρθρου με το function που φτιάξαμε στο βήμα 2
    soup = get_webpage_soup(article_url)
    # δημιουργία του λεξικού με τα δεδομένα του κάθε άρθρου με το function που φτιάξαμε στο βήμα 9
    articleD = get_article_elements_from_soup(soup, article_url)
    # προσθήκη του λεξικού με τα δεδομένα του κάθε άρθρου στη λίστα που δημιουργήσαμε στη αρχή
    all_articles_dataL.append(articleD)
    # καθυστέρηση ανάμεσα στα requests στο server
    delay = random.uniform(1, 3)
    time.sleep(delay)
  # δημιουργία dataframe από τη λίστα των λεξικών με τα δεδομένα κάθε άρθρου
  df = pd.DataFrame(all_articles_dataL)
  print(f"\nScraping completed. Fetshed {len(df)} articles.")
  return df

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
teasers_df = pd.read_csv('/content/drive/MyDrive/CSVFILES/kathimerini_ergatika_atiximata_teasersdf.csv')
teasers_df.head()

,Unnamed: 0,title,article_url,date,time,section,excerpt,author
0,0,Θεσσαλονίκη: Εργατικό ατύχημα στη Σίνδο – Τραυ...,https://www.kathimerini.gr/society/563648845/t...,2025-06-04,NaN,ΚΟΙΝΩΝΙΑ,Ο άνδρας μεταφέρθηκε στο «Παπανικολάου» σε σοβ...,NaN
1,1,Θεσσαλονίκη: Σοβαρό εργατικό ατύχημα – Ι.Χ. έπ...,https://www.kathimerini.gr/society/563596888/t...,2025-05-06,NaN,ΚΟΙΝΩΝΙΑ,Ενας εργαζόμενος σε έργα οδοποιίας τραυματίστη...,NaN
2,2,Ενα εργατικό ατύχημα ανά δύο ημέρες το πρώτο τ...,https://www.kathimerini.gr/economy/563585770/e...,2025-04-29,NaN,ΔΙΕΘΝΗΣ ΟΙΚΟΝΟΜΙΑ,Ραγδαία αύξηση παρουσιάζουν οι θάνατοι από εργ...,NaN
3,3,Ηράκλειο: Εργατικό ατύχημα σε επιχείρηση με μά...,https://www.kathimerini.gr/society/563521963/i...,2025-03-19,NaN,ΚΟΙΝΩΝΙΑ,"Ο 63χρονος ιδιοκτήτης συνελήφθη, ενώ η υπόθεση...",NaN
4,4,Εργατικό ατύχημα στη Θεσσαλονίκη: Γυναίκα έπεσ...,https://www.kathimerini.gr/society/563478304/e...,2025-02-20,NaN,ΚΟΙΝΩΝΙΑ,Η γυναίκα έχει τις αισθήσεις της ενώ έχει τραυ...,NaN


In [ ]:
len(teasers_df)

38

In [ ]:
articles_df=scrape_kathimerini_articles_pages(teasers_df)
articles_df

Processing rows: 100%|██████████| 38/38 [01:43<00:00,  2.72s/it]


Scraping completed. Fetshed 38 articles.


,article_url,title,date,time,section,author,excerpt,text_body,feat_img_link,feat_img_cation
0,https://www.kathimerini.gr/society/563648845/t...,Θεσσαλονίκη: Εργατικό ατύχημα στη Σίνδο – Τραυ...,2025-06-04,12:12:54,Κοινωνία,Newsroom,Ο άνδρας μεταφέρθηκε στο «Παπανικολάου» σε σοβ...,Ενας 34χρονος άνδρας τραυματίστηκε σοβαρά σε ε...,https://www.kathimerini.gr/wp-content/uploads/...,
1,https://www.kathimerini.gr/society/563596888/t...,Θεσσαλονίκη: Σοβαρό εργατικό ατύχημα – Ι.Χ. έπ...,2025-05-06,08:59:34,Κοινωνία,Newsroom,Ενας εργαζόμενος σε έργα οδοποιίας τραυματίστη...,Εργαζόμενος σε έργα οδοποιίας τραυματίστηκε σο...,https://www.kathimerini.gr/wp-content/uploads/...,Φωτ. αρχείου: Intime
2,https://www.kathimerini.gr/economy/563585770/e...,Ενα εργατικό ατύχημα ανά δύο ημέρες το πρώτο τ...,2025-04-29,08:08:00,Οικονομία,Ρούλα Σαλούρου,"Το διάστημα Ιανουαρίου – Απριλίου 2025, οι θάν...",Ραγδαία αύξηση παρουσιάζουν οι θάνατοι από εργ...,https://www.kathimerini.gr/wp-content/uploads/...,Φωτ. Shutterstock.
3,https://www.kathimerini.gr/society/563521963/i...,Ηράκλειο: Εργατικό ατύχημα σε επιχείρηση με μά...,2025-03-19,10:13:30,Κοινωνία,Newsroom,"Ο 63χρονος ιδιοκτήτης συνελήφθη, ενώ η υπόθεση...",Εργατικό ατύχημα σημειώθηκε σε περιοχή του Δήμ...,https://www.kathimerini.gr/wp-content/uploads/...,(φωτ. αρχείου INTIME)
4,https://www.kathimerini.gr/society/563478304/e...,Εργατικό ατύχημα στη Θεσσαλονίκη: Γυναίκα έπεσ...,2025-02-20,10:56:43,Κοινωνία,Newsroom,Η γυναίκα έχει τις αισθήσεις της ενώ έχει τραυ...,Σε εξέλιξη βρίσκεται στο Σταθμό Μεταφόρτωσης Α...,https://www.kathimerini.gr/wp-content/uploads/...,Φωτ. αρχείου: Shutterstock
5,https://www.kathimerini.gr/society/563395597/t...,Θεσσαλονίκη: Εργατικό ατύχημα στο λιμάνι με θύ...,2024-12-28,15:41:29,Κοινωνία,Newsroom,Ο τραυματίας διακομίστηκε με ασθενοφόρο του ΕΚ...,Ενας 60χρονος εργαζόμενος οδηγός φορτηγού οχήμ...,https://www.kathimerini.gr/wp-content/uploads/...,Φωτ. αρχείου: Intime
6,https://www.kathimerini.gr/society/563183485/g...,ΓΣΕΕ: Νέο εργατικό ατύχημα στο λιμάνι του Πειραιά,2024-08-22,14:52:09,Κοινωνία,Newsroom,Η Συνομοσπονδία ζητά την άμεση παρέμβαση των α...,"Για πολλοστή φορά, η Γενική Συνομοσπονδία Εργα...",https://www.kathimerini.gr/wp-content/uploads/...,Φωτ.: Intime
7,https://www.kathimerini.gr/society/562879141/e...,Εργατικό ατύχημα στα Χανιά – Πιάστηκε το χέρι ...,2024-02-12,13:33:17,Κοινωνία,Newsroom,Η Πυροσβεστική έσπευσε στο σημείο με τρία οχήμ...,Ενας 57χρονος τραυματίστηκε στα Χανιά ενώ εργα...,https://www.kathimerini.gr/wp-content/uploads/...,Φωτ. αρχείου: Intime
8,https://www.kathimerini.gr/society/562800598/e...,Εργατικό ατύχημα στη Δραπετσώνα: Τραυματίστηκε...,2023-12-24,16:05:45,Κοινωνία,Newsroom,Το ατύχημα σημειώθηκε κατά τη διάρκεια πρόσδεσ...,Από σπάσιμο του σχοινιού πρόσδεσης τραυματίστη...,https://www.kathimerini.gr/wp-content/uploads/...,Φωτ. αρχείου: Intime
9,https://www.kathimerini.gr/society/562648900/s...,Σαλαμίνα: Εργατικό ατύχημα σε ναυπηγείο,2023-10-02,20:55:45,Κοινωνία,Newsroom,Ενας 52χρονος εργάτης τραυματίστηκε σήμερα σε ...,Ενας 52χρονος εργάτης τραυματίστηκε σήμερα σε ...,https://www.kathimerini.gr/wp-content/uploads/...,Φωτ. αρχείου: Shutterstock


In [ ]:
articles_df.to_csv("/content/drive/MyDrive/CSVFILES/kathimerini_ergatika_atiximata_articlesdf.csv", index=False)